In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### CoronaVírus no Brasil

Os dados deste notebook são dados do governo federal.

https://covid.saude.gov.br/


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import unidecode

In [ ]:
dt = pd.read_csv('/kaggle/input/corona-virus-brazil/brazil_covid19.csv')
dt.head()

In [ ]:
dt.info()

In [ ]:
dt.date = pd.to_datetime(dt['date'])
dt.info()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
dt.describe()

In [ ]:
dt.columns = ["date", "regiao", "estado", "casos", "mortes"]
dt.head()

In [ ]:
dt[dt.casos == 1178075.00]

In [ ]:
dt["casos_diarios"] = dt.groupby('estado')['casos'].apply(lambda x: x.shift(1) - x)*-1
dt["mortes_diarias"] = dt.groupby('estado')['mortes'].apply(lambda x: x.shift(1) - x)*-1
dt.describe()

In [ ]:
mxdt = str(max(dt['date']))
print("dados de: ", min(dt['date']), max(dt['date']))

In [ ]:
dtsum = dt[['date', 'casos', 'mortes']].groupby('date').sum()
dtsum.head()

In [ ]:

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Bar(x=dtsum.index, y=dtsum.casos,  name="Casos Confirmados"), row = 1, col = 1)
fig.add_trace(go.Bar(x=dtsum.index, y=dtsum.mortes, name="Óbitos"), row = 2, col = 1)
fig.update_layout(title_text=("Situação até - "+mxdt), paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [ ]:
dtsum2 = dt[['date', 'casos_diarios', 'mortes_diarias']].groupby("date").sum()
dtsum2["media_casos_semanal"] = dtsum2[["casos_diarios"]].rolling(window=7).mean()
dtsum2["media_mortes_semanal"] = dtsum2[["mortes_diarias"]].rolling(window=7).mean()
dtsum2["media_casos_30dias"] = dtsum2[["casos_diarios"]].rolling(window=30).mean()
dtsum2["media_mortes_30dias"] = dtsum2[["mortes_diarias"]].rolling(window=30).mean()


In [ ]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Bar(x=dtsum2.index, y=dtsum2.casos_diarios,  name="Casos Confirmados diários", yaxis='y1'), row = 1, col = 1)
fig.add_trace(go.Scatter(x=dtsum2.index, y=dtsum2.media_casos_semanal,  name="Casos Confirmados - Média Semanal", yaxis='y2'),  row = 1, col = 1)


fig.add_trace(go.Bar(x=dtsum2.index, y=dtsum2.mortes_diarias, name="Óbitos diários", yaxis='y1'), row = 2, col = 1)
fig.add_trace(go.Scatter(x=dtsum2.index, y=dtsum2.media_mortes_semanal,  name="Óbitos - Média Semanal", yaxis='y2'), row = 2, col = 1)
fig.update_layout(title_text=("Situação diária até - "+mxdt), 
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [ ]:
fig = make_subplots(1, 1)

fig.add_trace(go.Scatter(x=dtsum2.index, y=dtsum2.media_casos_30dias,  name="Confirmados - Media 30 dias", yaxis='y1'), row = 1, col = 1)
fig.add_trace(go.Scatter(x=dtsum2.index, y=dtsum2.media_casos_semanal,  name="Confirmados - Média Semanal", yaxis='y2'),  row = 1, col = 1)

fig.update_layout(title_text=("Média de Confirmados até - "+mxdt), 
    plot_bgcolor='rgba(0,0,0,0)')

fig.show()

In [ ]:
dtsum["tx_letalidade"] = dtsum.mortes/dtsum.casos

fig = px.line(x=dtsum.index, y=dtsum.tx_letalidade,  title="Taxa de letalidade")
fig.update_layout(xaxis_title="Data",
    yaxis_title="Taxa de Letalidade", 
    plot_bgcolor='rgba(0,0,0,0)', yaxis_tickformat = '.2%')
fig.show()

In [ ]:
dtsum3 = dt.loc[(dt['date']== mxdt), ['estado', 'casos', 'mortes']]
dtsum3.columns = ['UF', 'casos', 'mortes']
dtsum3.head()

### Taxa de letalidade por UF em 2020-11-19

In [ ]:
dtsum3['% Taxa letalidade'] = round(dtsum3.mortes/dtsum3.casos*100, 2)
dtsum3[['UF', '% Taxa letalidade']].sort_values(by='% Taxa letalidade', ascending=False).style.background_gradient(cmap='Reds', subset = ['% Taxa letalidade'])

In [ ]:
dtpop = pd.read_csv('/kaggle/input/corona-virus-brazil/brazil_population_2019.csv', error_bad_lines=False)
dtpop.head()

In [ ]:
dtpopsum = dtpop[['region', 'state','population']].groupby(['state', 'region'], as_index = False).sum()
dtpopsum['Estado'] = dtpopsum['state'].apply(unidecode.unidecode)
dtpopsum.head()

In [ ]:
import geopandas as gpd
import json
import unidecode 

mapbrasil = gpd.read_file('/kaggle/input/majson/data/Brasil.json')
mapbrasil['Estado'] = mapbrasil['ESTADO'].apply(unidecode.unidecode)
mapbrasil.head()


**Data Frame População**

In [ ]:
dtsum4 = pd.merge(dtpopsum, mapbrasil, how= 'left')
dtsum4 = dtsum4[['UF', 'Estado','population']]
dtsum4.head()

### Situação da quantidade de casos e óbitos a cada 100mil habitantes até 30-11-2020


In [ ]:
dtsum5 = pd.merge(dtsum4, dtsum3, how = 'inner')
dtsum5['caso100mil'] = dtsum5['casos']/dtsum5['population']*100000
dtsum5['caso100mil'] = dtsum5['caso100mil'].astype('int64')
dtsum5['mortes100mil'] = dtsum5['mortes']/dtsum5['population']*100000
dtsum5['mortes100mil'] = dtsum5['mortes100mil'].astype('int64')
dtsum5[['Estado', 'caso100mil', 'mortes100mil']].sort_values(by='caso100mil', ascending=False).style.background_gradient(cmap='Reds', subset = ["caso100mil", 'mortes100mil'])

In [ ]:
populacao = dtpop.population.sum()
dtsum['populacao'] = populacao
dtsum['casos100mil'] = dtsum.casos/dtsum.populacao*100000
dtsum['casos100mil'] = dtsum['casos100mil'].astype('int64')

dtsum['mortes100mil'] = dtsum.mortes/dtsum.populacao*100000
dtsum['mortes100mil'] = dtsum['mortes100mil'].astype('int64')

fig = px.bar(dtsum, x = dtsum.index, y = 'casos100mil', title = "Casos por 100mil habitantes - Brasil")
fig.update_layout( plot_bgcolor='rgba(0,0,0,0)', height = 400)
fig.show()

In [ ]:
fig = px.bar(dtsum, x = dtsum.index, y = 'mortes100mil', title = "Óbitos por 100mil habitantes - Brasil")
fig.update_layout( plot_bgcolor='rgba(0,0,0,0)', height = 400)
fig.show()

In [ ]:
estado = np.unique(dt[['estado', 'regiao']].values)
regiao = np.unique(dt[['regiao']].values)

dt_ufreg = dt[['estado', 'regiao']].drop_duplicates()
dt_ufreg.columns = ['Estado', 'regiao']


In [ ]:
dtreg = dt[['date', 'regiao', 'casos', 'mortes']].groupby(['date', 'regiao'], as_index=False).sum()

dtpopreg = dtpopsum[['region', 'population']].groupby('region', as_index=False).sum()
dtpopreg.columns = ['regiao', 'population']
dtreg2 = pd.merge(dtreg, dtpopreg, how='left')
dtreg2.head()


In [ ]:
dtreg2['caso100mil'] = dtreg2['casos']/dtreg2['population']*100000
dtreg2['caso100mil'] = dtreg2['caso100mil'].astype('int64')
dtreg2['morte100mil'] = dtreg2['mortes']/dtreg2['population']*100000
dtreg2['morte100mil'] = dtreg2['morte100mil'].astype('int64')
dtreg2['txletalidade'] = dtreg2['mortes']/dtreg2['casos']
dtreg2.head()

fig = px.scatter(dtreg2, x ='date', y='caso100mil', color='regiao', title = 'Casos 100mil habitantes por Região')
fig.update_layout( plot_bgcolor='rgba(0,0,0,0)', height = 500)
fig.show()

In [ ]:
fig = px.scatter(dtreg2, x ='date', y='morte100mil', color='regiao', title = 'Mortes 100mil habitantes por Região')
fig.update_layout( plot_bgcolor='rgba(0,0,0,0)', height = 500)
fig.show()

In [ ]:
dtreg2 = dtreg2.reset_index()
dtreg2.head()

In [ ]:
fig = px.scatter(dtreg2, x ='date', y='casos', color='regiao', title = 'Casos por região')
fig.update_layout( plot_bgcolor='rgba(0,0,0,0)', height = 500)
fig.show()

In [ ]:
fig = px.scatter(dtreg2, x ='date', y='mortes', color='regiao', title = 'Mortes por região')
fig.update_layout( plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [ ]:
dtreg2["casos_var"] = dtreg2['casos'] - dtreg2.groupby('regiao')['casos'].shift(1)
dtreg2["mortes_var"] = dtreg2['mortes'] - dtreg2.groupby('regiao')['mortes'].shift(1)

dtreg2["casos7dias"] = dtreg2.groupby('regiao')['casos_var'].shift(7)
dtreg2["mortes7dias"] =  dtreg2.groupby('regiao')['mortes_var'].shift(7)

dtreg2["casos14dias"] = dtreg2.groupby('regiao')['casos_var'].shift(14)
dtreg2["casos21dias"] = dtreg2.groupby('regiao')['casos_var'].shift(21)
dtreg2["casos30dias"] = dtreg2.groupby('regiao')['casos_var'].shift(30)
dtreg2["mortes14dias"] = dtreg2.groupby('regiao')['mortes_var'].shift(14)

dtreg2.tail(151)

In [ ]:
fig = px.line(dtreg2, x ='date', y='casos_var', color='regiao', title = 'Casos por região')
fig.update_layout( plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [ ]:
fig = px.line(dtreg2, x ='date', y='mortes_var', color='regiao', title = 'Mortes por região')
fig.update_layout( plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Mortes x Casos 7 dias atras", "Mortes x Casos 14 dias atras", "Mortes x Casos 21 dias atras", "Mortes x Casos 30 dias atras"))


fig.add_trace(go.Scatter(x =dtreg2['casos7dias'], y=dtreg2['mortes'], mode = "markers"), row = 1, col = 1)
fig.add_trace(go.Scatter(x =dtreg2['casos14dias'], y=dtreg2['mortes'], mode = "markers"), row = 2, col = 1)
fig.add_trace(go.Scatter(x =dtreg2['casos21dias'], y=dtreg2['mortes'], mode = "markers"), row = 1, col = 2)
fig.add_trace(go.Scatter(x =dtreg2['casos30dias'], y=dtreg2['mortes'], mode = "markers"), row = 2, col = 2)
fig.update_layout( plot_bgcolor='rgba(0,0,0,0)', height=700)
fig.show()

In [ ]:
dtRG_CO = dt[dt["regiao"]=="Centro-Oeste"]
dtRG_CO['txletalidade'] = dtRG_CO["mortes"]/dtRG_CO["casos"]

dtRG_CO.head()

In [ ]:
fig = px.line(dtRG_CO, x ='date', y='txletalidade', color='estado', title = 'Taxa de letalidade Centro Oeste')
fig.update_layout(xaxis_title="Data",
    yaxis_title="Taxa de Letalidade", 
    plot_bgcolor='rgba(0,0,0,0)', yaxis_tickformat = '.2%')
fig.show()

In [ ]:
dtRG_CO = dt[dt["regiao"]=="Sudeste"]
dtRG_CO['txletalidade'] = dtRG_CO["mortes"]/dtRG_CO["casos"]


In [ ]:
fig = px.line(dtRG_CO, x ='date', y='txletalidade', color='estado', title = 'Taxa de letalidade por Região Sudeste')
fig.update_layout(xaxis_title="Data",
    yaxis_title="Taxa de Letalidade", 
    plot_bgcolor='rgba(0,0,0,0)', yaxis_tickformat = '.2%')
fig.show()

In [ ]:
fig = px.line(dtreg2, x ='date', y='txletalidade', color='regiao', title = 'Taxa de letalidade por Região')
fig.update_layout(xaxis_title="Data",
    yaxis_title="Taxa de Letalidade", 
    plot_bgcolor='rgba(0,0,0,0)', yaxis_tickformat = '.2%')
fig.show()

In [ ]:
dtsum.head()

In [ ]:

dt2 = dtsum.loc[dtsum['casos']!=0, ['casos']]
#dt2.drop(['date'],axis=1,inplace = True)
dt2.reset_index(drop=True, inplace=True)
dt2['dias'] = dt2.index
dt2['dias'] = dt2['dias'].apply(lambda x: x+1)
dt2.head()

In [ ]:
txCasos = [1]
txCasos[-1]

In [ ]:
def casos_Taxa(taxa, dias):
    txCasos = [1]
    for i in range(len(listDias)-1):
        
        txCasos.append(taxa*txCasos[-1])
        
    return txCasos

listDias = list(range(0, max(dt2['dias']),2))
dt_diasSup = pd.DataFrame({'dias_real':listDias, 'dias2x':casos_Taxa(2, listDias), 'dias3x':casos_Taxa(3, listDias), 'dias5x':casos_Taxa(5, listDias)})
dt_diasSup.head()

In [ ]:

fig = go.Figure(data=[
    go.Scatter(x=dt2.index, y=dt2['casos'], name='Casos Reais',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias2x'], name = '2x',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias3x'], name = '3x',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias5x'], name = '5x',mode="lines+markers")
])

fig.update_layout( paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title= "Crescimento de casos a cada 2 dias",
    xaxis_title="Dias",
    yaxis_title="Quantidade (log)",
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 5
    ),
    yaxis_type="log")

fig.show()

In [ ]:
listDias = list(range(0, max(dt2['dias']),3))
dt_diasSup = pd.DataFrame({'dias_real':listDias, 'dias1.5x':casos_Taxa(1.5, listDias), 'dias2x':casos_Taxa(2, listDias), 'dias3x':casos_Taxa(3, listDias), 'dias5x':casos_Taxa(5, listDias)})


fig = go.Figure(data=[
    go.Scatter(x=dt2.index, y=dt2['casos'], name='Casos Reais',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias1.5x'], name = '1.5x',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias2x'], name = '2x',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias3x'], name = '3x',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias5x'], name = '5x',mode="lines+markers")
    
])

fig.update_layout( paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title= "Crescimento de casos a cada 3 dias",
    xaxis_title="Dias",
    yaxis_title="Quantidade (log)",
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 5
    ),
    yaxis_type="log")

fig.show()


In [ ]:
listDias = list(range(0, max(dt2['dias']),5))
dt_diasSup = pd.DataFrame({'dias_real':listDias, 'dias1.5x':casos_Taxa(1.5, listDias), 'dias2x':casos_Taxa(2, listDias), 'dias3x':casos_Taxa(3, listDias), 'dias5x':casos_Taxa(5, listDias)})


fig = go.Figure(data=[
    go.Scatter(x=dt2.index, y=dt2['casos'], name='Casos Reais',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias1.5x'], name = '1.5x',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias2x'], name = '2x',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias3x'], name = '3x',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias5x'], name = '5x',mode="lines+markers")
    
])

fig.update_layout( paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title= "Crescimento de casos a cada 5 dias",
    xaxis_title="Dias",
    yaxis_title="Quantidade (log)",
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 5
    ),
    yaxis_type="log")

fig.show()


In [ ]:
listDias = list(range(0, max(dt2['dias']),7))
dt_diasSup = pd.DataFrame({'dias_real':listDias, 'dias1.5x':casos_Taxa(1.5, listDias), 'dias2x':casos_Taxa(2, listDias), 'dias3x':casos_Taxa(3, listDias), 'dias5x':casos_Taxa(5, listDias)})


fig = go.Figure(data=[
    go.Scatter(x=dt2.index, y=dt2['casos'], name='Casos Reais',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias1.5x'], name = '1.5x',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias2x'], name = '2x',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias3x'], name = '3x',mode="lines+markers"),
    go.Scatter(x=dt_diasSup ['dias_real'], y=dt_diasSup ['dias5x'], name = '5x',mode="lines+markers")
    
])

fig.update_layout( paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title= "Crescimento de casos a cada 7 dias",
    xaxis_title="Dias",
    yaxis_title="Quantidade (log)",
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 5
    ),
    yaxis_type="log")

fig.show()
